<a href="https://colab.research.google.com/github/MIKEPY101/End-to-End-Machine-Learning-Project/blob/master/Basic_convnet_It%E2%80%99s_a_stack_of_Conv2D_and_MaxPooling2D_layers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Basic convnet. It’s a stack of Conv2D and MaxPooling2D layers

In [9]:
# Instantiating a small convnet
from keras import layers
from keras import models
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))

### A convnet takes as input tensors of shape (image_height, image_width, image_channels) (not including the batch dimension). In this case, we’ll configure the convnet to process inputs of size (28, 28, 1), which is the format of MNIST images. We’ll do this by passing the argument input_shape=(28, 28, 1) to the first layer. Let’s display the architecture of the convnet so far:

In [10]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_12 (Conv2D)           (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 3, 3, 64)          36928     
Total params: 55,744
Trainable params: 55,744
Non-trainable params: 0
_________________________________________________________________


### You can see that the output of every Conv2D and MaxPooling2D layer is a 3D tensor of shape (height, width, channels). The width and height dimensions tend to shrink as you go deeper in the network. 

### The number of channels is controlled by the first argument passed to the Conv2D layers (32 or 64). 

### The next step is to feed the last output tensor (of shape (3, 3, 64)) into a densely connected classifier network; a stack of Dense layers. These classifiers process vectors, which are 1D, whereas the current output is a 3D tensor. 

### First we have to flatten the 3D outputs to 1D, and then add a few Dense layers on top.

In [11]:
# Adding a classifier on top of the convnet
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

### We’ll do 10-way classification, using a final layer with 10 outputs and a softmax activation.

In [6]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 3, 3, 64)          36928     
_________________________________________________________________
flatten (Flatten)            (None, 576)               0         
_________________________________________________________________
dense (Dense)                (None, 64)               

### The (3, 3, 64) outputs are flattened into vectors of shape (576,) before going through two Dense layers. Now, let’s train the convnet on the MNIST digits.

In [12]:
# Training the convnet on MNIST images
from keras.datasets import mnist
from keras.utils import to_categorical
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()
train_images = train_images.reshape((60000, 28, 28, 1))
train_images = train_images.astype('float32') / 255
test_images = test_images.reshape((10000, 28, 28, 1))
test_images = test_images.astype('float32') / 255
train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)
model.compile(optimizer='rmsprop',
loss='categorical_crossentropy',
metrics=['accuracy'])
model.fit(train_images, train_labels, epochs=5, batch_size=64)

Epoch 1/5
938/938 [==============================] - 49s 52ms/step - loss: 0.1705 - accuracy: 0.9469
Epoch 2/5
938/938 [==============================] - 47s 50ms/step - loss: 0.0463 - accuracy: 0.9858
Epoch 3/5
938/938 [==============================] - 47s 50ms/step - loss: 0.0322 - accuracy: 0.9897
Epoch 4/5
938/938 [==============================] - 47s 50ms/step - loss: 0.0239 - accuracy: 0.9923
Epoch 5/5
938/938 [==============================] - 47s 50ms/step - loss: 0.0195 - accuracy: 0.9942


In [13]:
# Let’s evaluate the model on the test data:
test_loss, test_acc = model.evaluate(test_images, test_labels)
test_acc


313/313 [==============================] - 3s 9ms/step - loss: 0.0353 - accuracy: 0.9892


0.9891999959945679